## Imports
We load NumPy for array math, pandas for tables, Matplotlib for production charts, SciPy's stats module for reliability quantiles, and CVXPY for solving the convex production planning problem.

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import cvxpy as cp
from typing import Dict, Any, Tuple

## Production System Class
This class randomly generates a fleet of convex generators. Parameters: `n_machines` controls fleet size, `seed` ensures reproducibility, `alpha/beta/gamma_bounds` define the cost coefficients, and `capacity_bounds` set min/max outputs. Methods `describe()` and `total_capacity()` summarize the fleet.

In [22]:
class ProductionSystem:
    """
    Factory model holding production machine cost curves and capacity limits.

    Each machine i has cost: C_i(x_i) = alpha_i x_i^2 + beta_i x_i + gamma_i
    with 0 <= x_i <= u_i (minimum assumed zero).
    """
    def __init__(self, n_machines: int = 5, seed: int = 42,
                 alpha_bounds: Tuple[float, float] = (0.05, 0.2),
                 beta_bounds: Tuple[float, float] = (2.0, 5.0),
                 gamma_bounds: Tuple[float, float] = (10.0, 20.0),
                 capacity_bounds: Tuple[float, float] = (50.0, 150.0)):
        rng = np.random.default_rng(seed)
        self.n = n_machines
        # Quadratic production cost curvature
        self.alpha = rng.uniform(alpha_bounds[0], alpha_bounds[1], n_machines)
        # Linear production cost slope
        self.beta = rng.uniform(beta_bounds[0], beta_bounds[1], n_machines)
        # Fixed cost component
        self.gamma = rng.uniform(gamma_bounds[0], gamma_bounds[1], n_machines)
        # Lower/upper production capacity (units)
        self.l = np.zeros(n_machines)
        self.u = rng.uniform(capacity_bounds[0], capacity_bounds[1], n_machines)

    def describe(self) -> pd.DataFrame:
        machines = np.arange(1, self.n + 1)
        data = {
            "alpha": self.alpha,
            "beta": self.beta,
            "gamma": self.gamma,
            "min_cap": self.l,
            "max_capacity": self.u,
        }
        return pd.DataFrame(data, index=machines).rename_axis("production_machine")

    def total_capacity(self) -> float:
        return float(np.sum(self.u))

## Production Planning Solver
`solve_dispatch` enforces a reliability-adjusted customer demand. Inputs: `system` (factory), `mu_D`/`sigma_D` (demand stats), `reliability` (service level), and `mode` (Normal vs Chebyshev). It builds CVXPY variable `x` (production units), computes effective demand `D_eff`, sets up quadratic production cost objective, enforces capacity + demand constraints, solves, and returns primal/dual economic signals.

In [23]:
def solve_dispatch(
    system: ProductionSystem,
    mu_D: float,
    sigma_D: float,
    reliability: float = 0.95,
    mode: str = "normal",
) -> Dict[str, Any]:
    """Solve the stochastic production planning problem under Normal or Chebyshev reliability buffering.

    We convert a probabilistic customer demand requirement into a deterministic
    effective demand D_eff then solve:
        minimize   sum_i (alpha_i x_i^2 + beta_i x_i)
        subject to sum_i x_i >= D_eff
                   0 <= x_i <= max_capacity_i
    Fixed costs gamma_i are added after solving for reporting.

    Returns dictionary with optimal production vector, total cost (incl. gamma),
    effective demand enforced, and dual multipliers:
      lambda : shadow price of 1 additional unit of required demand
      nu_u   : scarcity indicator per machine at its capacity
      nu_l   : lower bound duals (typically zero here)
    """
    if sigma_D < 0 or mu_D < 0:
        raise ValueError("Demand statistics must be non-negative")
    if not 0.0 < reliability < 1.0:
        raise ValueError("Reliability must lie strictly between 0 and 1")

    x = cp.Variable(system.n)
    alpha_risk = 1.0 - reliability

    if mode == "normal":
        z_score = stats.norm.ppf(reliability)
        D_eff = mu_D + z_score * sigma_D
    elif mode == "robust":
        k_robust = np.sqrt((1 - alpha_risk) / alpha_risk)
        D_eff = mu_D + k_robust * sigma_D
    else:
        raise ValueError("Mode must be 'normal' or 'robust'")

    if D_eff > system.total_capacity() + 1e-6:
        raise ValueError("Effective customer demand exceeds installed production capacity")

    objective = cp.Minimize(
        cp.sum(cp.multiply(system.alpha, x**2) + cp.multiply(system.beta, x))
    )

    c_demand = [cp.sum(x) >= D_eff]
    c_upper = [x <= system.u]
    c_lower = [x >= system.l]
    constraints = c_demand + c_upper + c_lower

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.OSQP, warm_start=True)

    if prob.status not in {"optimal", "optimal_inaccurate"}:
        return {"status": prob.status}

    dispatch = x.value
    total_cost = float(
        np.sum(system.alpha * dispatch**2 + system.beta * dispatch + system.gamma)
    )

    return {
        "status": prob.status,
        "x": dispatch,
        "cost": total_cost,
        "objective": prob.value,
        "D_eff": float(D_eff),
        "lambda": c_demand[0].dual_value,
        "nu_u": c_upper[0].dual_value,
        "nu_l": c_lower[0].dual_value,
    }

## KKT Verification
`verify_kkt` receives the system and solver results, computes the stationarity residual (`grad_f - λ + ν⁺ - ν⁻`) and complementary slackness (`(∑x - D_eff) * λ`), prints both, and reports whether they are below tolerance.

In [24]:
def verify_kkt(system: ProductionSystem, res: Dict[str, Any], tol: float = 1e-4) -> Dict[str, float]:
    """Compute stationarity and complementary slackness residuals."""
    
    if res.get("status") not in {"optimal", "optimal_inaccurate"}:
        print("Cannot verify KKT: solution unavailable.")
        return {}
    
    dispatch = res["x"]
    lam = res["lambda"]
    nu_u = res["nu_u"]
    nu_l = res["nu_l"]
    
    grad_f = 2 * system.alpha * dispatch + system.beta
    stationarity_residual = grad_f - lam + nu_u - nu_l
    max_stationarity = float(np.max(np.abs(stationarity_residual)))
    
    total_prod = float(np.sum(dispatch))
    slack = total_prod - res["D_eff"]
    comp_slack_error = float(abs(slack * lam))
    
    print("\n--- KKT CHECK ---")
    print(f"Stationarity residual: {max_stationarity:.2e}")
    print(f"Complementary slackness residual: {comp_slack_error:.2e}")
    
    if max_stationarity < tol and comp_slack_error < tol:
        print("KKT conditions satisfied within tolerance.")
    else:
        print("Warning: residuals exceed tolerance.")
    
    return {"stationarity": max_stationarity, "slackness": comp_slack_error}

In [25]:
# 1. Create the factory
my_factory = ProductionSystem(n_machines=5)

# 2. Define the customer demand scenario
MEAN_DEMAND = 300
SIGMA = 30
RELIABILITY = 0.95  # 95% service level

# 3. Run the Normal (Gaussian) buffering model
print("--- NORMAL MODEL (Gaussian demand) ---")
res_normal = solve_dispatch(
    my_factory, MEAN_DEMAND, SIGMA, reliability=RELIABILITY, mode="normal"
)
print(f"Total Production Cost: ${res_normal['cost']:,.2f}")
print(f"Effective required production (D_eff): {res_normal['D_eff']:.2f} units")

# 4. Run the Robust (Chebyshev) buffering model
print("\n--- ROBUST MODEL (Distribution-free) ---")
res_robust = solve_dispatch(
    my_factory, MEAN_DEMAND, SIGMA, reliability=RELIABILITY, mode="robust"
)
print(f"Total Production Cost: ${res_robust['cost']:,.2f}")
print(f"Effective required production (D_eff): {res_robust['D_eff']:.2f} units")

# 5. Verify optimality (Normal model)
verify_kkt(my_factory, res_normal)

--- NORMAL MODEL (Gaussian demand) ---
Total Production Cost: $4,305.68
Effective required production (D_eff): 349.35 units

--- ROBUST MODEL (Distribution-free) ---
Total Production Cost: $6,336.46
Effective required production (D_eff): 430.77 units

--- KKT CHECK ---
Stationarity residual: 3.52e-13
Complementary slackness residual: 1.23e-12
KKT conditions satisfied within tolerance.


{'stationarity': 3.5216274341109965e-13, 'slackness': 1.2315451736330222e-12}

## Production Summary Table
`summarize_dispatch` merges machine parameters with solution outputs: production units, slacks to bounds, dual multipliers, marginal production cost, and each machine's total production cost share.

In [26]:
def summarize_dispatch(system, res):
    if res.get("status") not in {"optimal", "optimal_inaccurate"}:
        raise ValueError("Cannot summarize infeasible solution")

    production = res["x"]
    table = system.describe().copy()
    table["production"] = production
    table["upper_slack"] = system.u - production
    table["lower_slack"] = production - system.l
    table["nu_upper"] = res["nu_u"]
    table["nu_lower"] = res["nu_l"]
    table["marginal_cost"] = 2 * system.alpha * production + system.beta
    table["incremental_cost"] = system.alpha * production**2 + system.beta * production + system.gamma
    return table

## Scenario Comparison
We instantiate a 6-machine factory, set customer demand statistics (`mu_D`, `sigma_D`) and service level, then solve under both Gaussian (`normal`) and Chebyshev (`robust`) assumptions. For each mode we store results, show effective required production, total production cost, a preview of the production table, and run KKT checks.

In [27]:
# Scenario comparison for Gaussian vs. Chebyshev buffering
system = ProductionSystem(n_machines=6, seed=7)
mu_D = 350.0
sigma_D = 40.0
reliability = 0.95

print("Production machine parameters (cost coefficients & capacities):")
print(system.describe())

results = {}
summaries = {}
for mode in ("normal", "robust"):
    print(f"\n=== {mode.title()} model ===")
    res = solve_dispatch(system, mu_D, sigma_D, reliability=reliability, mode=mode)
    results[mode] = res
    summary = summarize_dispatch(system, res)
    summaries[mode] = summary
    print(f"Effective required production: {res['D_eff']:.1f} units")
    print(f"Total production cost: ${res['cost']:.2f}")
    print(summary[["production", "max_capacity", "marginal_cost"]])
    verify_kkt(system, res)

Production machine parameters (cost coefficients & capacities):
                       alpha      beta      gamma  min_cap  max_capacity
production_machine                                                      
1                   0.143764  2.015796  12.548696      0.0    112.217923
2                   0.184582  4.463685  14.450763      0.0    148.896015
3                   0.166353  4.391208  15.045483      0.0     71.530870
4                   0.083781  3.403805  15.534974      0.0     66.021203
5                   0.095025  2.909097  19.955003      0.0    111.253960
6                   0.181033  2.835277  17.926619      0.0     54.394201

=== Normal model ===
Effective required production: 415.8 units
Total production cost: $5359.32
                    production  max_capacity  marginal_cost
production_machine                                         
1                    75.517454    112.217923      23.729227
2                    52.186926    148.896015      23.729227
3              

In [ ]:
# Per-machine cost curves with chosen production points (Normal vs Robust)
import numpy as np
import matplotlib.pyplot as plt

# Ensure prerequisite objects exist: `system`, `results`
if 'system' not in globals() or 'results' not in globals():
    raise RuntimeError("Factory system or results dictionary is not defined. Run the scenario comparison cells first.")
if 'normal' not in results or 'robust' not in results:
    raise RuntimeError("Expected 'normal' and 'robust' keys in results. Run the comparison loop cell.")

x_plot = np.linspace(0, system.u.max() * 1.05, 200)
plt.figure(figsize=(9, 5))
for i in range(system.n):
    alpha_i = system.alpha[i]
    beta_i = system.beta[i]
    gamma_i = system.gamma[i]
    cost_curve = alpha_i * x_plot**2 + beta_i * x_plot + gamma_i
    plt.plot(x_plot, cost_curve, label=f"Machine {i+1} cost curve", linewidth=1.2)

# Production points for each reliability modeling mode
prod_normal = results['normal']['x']
prod_robust = results['robust']['x']
cost_normal = system.alpha * prod_normal**2 + system.beta * prod_normal + system.gamma
cost_robust = system.alpha * prod_robust**2 + system.beta * prod_robust + system.gamma

plt.scatter(prod_normal, cost_normal, marker='o', color='steelblue', edgecolor='black', s=60, label='Normal production')
plt.scatter(prod_robust, cost_robust, marker='x', color='firebrick', s=80, label='Robust production')

plt.title("Per-machine cost curves and chosen production points")
plt.xlabel("Production (Units)")
plt.ylabel("Cost ($)")
plt.legend(ncol=2, fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# Demand vs Total Cost plot (Normal vs Robust)
import numpy as np
import matplotlib.pyplot as plt

# Preconditions
if 'system' not in globals():
    raise RuntimeError("'system' not found. Run the scenario comparison cell to create the ProductionSystem instance.")
if 'sigma_D' not in globals() or 'reliability' not in globals():
    raise RuntimeError("Expected 'sigma_D' and 'reliability' variables from earlier cells.")

sigma = sigma_D  # demand standard deviation (held constant)
rel = reliability

# Safety factors
z_score = stats.norm.ppf(rel)
k_robust = np.sqrt(rel / (1 - rel))  # Cantelli one-sided bound factor

# Maximum feasible mean demand for both modes (keep both solvable)
cap = system.total_capacity()
max_mu_normal = cap - z_score * sigma
max_mu_robust = cap - k_robust * sigma
max_mu = min(max_mu_normal, max_mu_robust)

# Choose a start demand (>=0) and build range in steps of 50 units
step = 50
start_mu = max(0, step) if max_mu >= step else 0
# Ensure at least one point
if start_mu > max_mu:
    raise ValueError("No feasible demand range given current buffer parameters and capacity.")

mu_values = np.arange(start_mu, max_mu + 1e-9, step)

normal_costs = []
robust_costs = []
normal_eff_demands = []
robust_eff_demands = []

for mu in mu_values:
    res_n = solve_dispatch(system, mu, sigma, reliability=rel, mode='normal')
    res_r = solve_dispatch(system, mu, sigma, reliability=rel, mode='robust')
    normal_costs.append(res_n['cost'])
    robust_costs.append(res_r['cost'])
    normal_eff_demands.append(res_n['D_eff'])
    robust_eff_demands.append(res_r['D_eff'])

plt.figure(figsize=(9,5))
plt.plot(mu_values, normal_costs, marker='o', label='Normal mode cost', color='steelblue')
plt.plot(mu_values, robust_costs, marker='s', label='Robust mode cost', color='firebrick')
plt.xlabel('Mean Demand (Units)')
plt.ylabel('Total Production Cost ($)')
plt.title('Demand vs Total Cost — Normal vs Robust')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Optional: show a small summary table
import pandas as pd
summary_df = pd.DataFrame({
    'mean_demand': mu_values,
    'normal_cost': normal_costs,
    'robust_cost': robust_costs,
    'normal_D_eff': normal_eff_demands,
    'robust_D_eff': robust_eff_demands,
})
print("Sweep summary (first few rows):")
print(summary_df.head())